# Task A - MongoDB Data Model

MongoDB is a document-oriented database. The most important challenge in mongoDB is designing a suitable data model. Data Models are designed to balance the needs of the application and to check the performance charactersitics of database engine. The most important factors need to be considered while designing data models are application's usage of data and inherent structure of data. Hence, the key decision in **designing a data model is to structure the document according to the given data and how the application represents the relationships between data**. In MongoDB, documents can be created in two ways. They are embedded data model and normalized(referencing) data model. The difference between them is how the application is representing the data relationships. 

If the dataset is really huge, it is good to use reference data model, but in our given case, the data is not really huge and we think embedded data model is good to use. Let's see why. In embedded data model, we can store related data into one single document which **reduces the redundancy of data**, when compared to reference data model. Also, this allows us to **query efficiently and requires fewer updates and data processing**. In the given datasets, climate date attribute is unique, and for each day, there might or might not have several fire incidents happened. Since these two datasets are not so huge, combining them and using only one collection would be much easier to understand and find the related data info. Hence, we decided to use embedded data model.

An example of the desired embedded data model should look like:
```
{'_id': ObjectId('5cdeab0893436941e37eacfb'),
 'air_temperature_celcius': 20,
 'date': '5/03/2017',
 'hotspot': [],
 'max_wind_speed': 15.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.8,
 'station': 948701,
 'windspeed_knots': 11.5}
{'_id': ObjectId('5cdeab0893436941e37eacfc'),
 'air_temperature_celcius': 20,
 'date': '6/03/2017',
 'hotspot': [{'confidence': 87,
              'datetime': '2017-03-06T05:06:30',
              'latitude': -34.3795,
              'longitude': 141.6331,
              'surface_temperature_celcius': 62},
             {'confidence': 85,
              'datetime': '2017-03-06T05:06:20',
              'latitude': -34.3735,
              'longitude': 141.6604,
              'surface_temperature_celcius': 59}],
 'max_wind_speed': 21.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 52.9,
 'station': 948701,
 'windspeed_knots': 17.0}
```
In the above example, there are two different object types. The first object represents that there is no fire(hotspot) data for the climate date and the second object represents that there are two fire incidents occured on that day.

# Task B - Querying MongoDB using PyMongo

## Question 1

In this task, the data is read from csv files and loaded into the newly created mongoDB database. The collection in the database is structured as embedded data model. 

## Importing libraries

In [ ]:
# importing required packages 
import pandas as pd
import json
import pymongo
from pymongo import MongoClient
from pprint import pprint

## Loading and analysing data

In [ ]:
# reading csv files into dataframe using pandas
climate = pd.read_csv('climate_historic.csv')
hotspot = pd.read_csv('hotspot_historic.csv')

In [ ]:
# detailed information of climate csv
climate.info()

In [ ]:
#displaying column names 
print(climate.columns)
print(hotspot.columns)

In [ ]:
# detailed information of hotspot csv
hotspot.info()

In [ ]:
#printing first 5 rows of climate
climate.head()

In [ ]:
#printing first 5 rows of hotspot
hotspot.head()

## Connection to MongoDB

In [ ]:
# Connect on the default host and port
client = MongoClient()

# getting database to connect
db = client.fit5148_assignment_db

In [ ]:
#printing database names
print(client.list_database_names())

## Creation of documents (Embedded data model)

In [ ]:
# list to store all the documents
list_final = []
#iterating by each row in climate table
for index,row in climate.iterrows():
    #Structuring the data model
    final_dic = dict() #store the climate data into dictionary
    final_dic['station'] = row[0]
    final_dic['air_temperature_celcius'] = row[2]
    final_dic['relative_humidity'] = row[3]
    final_dic['windspeed_knots'] = row[4]
    final_dic['max_wind_speed'] = row[5]
    final_dic['precipitation'] = row[6]
    final_dic['date'] = row[1]
    
    #find the corresponding hotspot data for specific climate date
    hotspot_sub = hotspot[hotspot['date']==row[1]]
    hotspot_dicts = [] #store the hotspot data for each climate date
    # iterating by each row in hotspot subset table
    for index,row in hotspot_sub.iterrows():
        dic2 = dict()
        dic2['latitude'] = row[0]
        dic2['longitude'] = row[1]
        dic2['datetime'] = row[2]
        dic2['confidence'] = row[3]
        dic2['surface_temperature_celcius'] = row[5]
        hotspot_dicts.append(dic2)
    final_dic['hotspot'] = hotspot_dicts #assign hotspot data to dictionary
    list_final.append(final_dic) #append each document to a list

In [ ]:
#printing one document to check 
list_final[0:2]

## Insertion of documents into MongoDB

In [ ]:
#create a collection 
clim_hotspot = db.ClimateHostspot

In [ ]:
#dropping if there is any data in collection
clim_hotspot.drop()

In [ ]:
#inserting documents into the collection
clim_hotspot.insert_many(list_final)

In [ ]:
#counting the number of documents in the collection
clim_hotspot.find().count()

In [ ]:
#printing documents in a pretty manner using pprint package from the database
documents = clim_hotspot.find({})
for doc in documents:
    pprint(doc)

## Question-2

## a) Finding climate data on 10th Dec 2017

In [ ]:
data = clim_hotspot.find({ "date": "10/12/2017" },{"_id": 0, "hotspot": 0})
for rec in data:
    pprint(rec)

## b) Finding the latitude, longitude, surface temp, confidence when surface temp is between 65 and 100 

### Surface temp values of both 65 and 100 are included in the query search

In [ ]:
data = clim_hotspot.aggregate([{"$unwind":"$hotspot"},
                                {"$match":{"$and":[{"hotspot.surface_temperature_celcius":{"$gte":65}},
                                                  {"hotspot.surface_temperature_celcius":{"$lte":100}}]}},
                                {"$project":{"_id":0,"hotspot.latitude":1,"hotspot.longitude":1,
                                             "hotspot.surface_temperature_celcius":1,"hotspot.confidence":1}}
                               ])
for rec in data:
    pprint(rec)

## c) Finding date, surface temp, air temp, max wind speed, relative humidity on 15th and 16th Dec 2017

In [ ]:
data = clim_hotspot.find({"$or": [{"date": "15/12/2017"},{"date":"16/12/2017"}] },
                          {"_id": 0, "date":1,"hotspot.surface_temperature_celcius": 1,"air_temperature_celcius":1,
                          "relative_humidity":1,"max_wind_speed":1})
for rec in data:
    pprint(rec)

## d) Finding air temp, confidence, datetime, surface temp when confidence is between 80 and 100. 

### Both confidence values 80 and 100 are included in the query search

In [ ]:
data = clim_hotspot.aggregate([{"$unwind":"$hotspot"},
                                {"$match":{"$and":[{"hotspot.confidence":{"$gte":80}},
                                                  {"hotspot.confidence":{"$lte":100}}]}},
                                {"$project":{"_id":0,"hotspot.datetime":1,"air_temperature_celcius":1,
                                             "hotspot.surface_temperature_celcius":1,"hotspot.confidence":1}}
                               ])
for rec in data:
    pprint(rec)

## e) Finding the top 10 records for highest surface temperature

In [ ]:
data = clim_hotspot.aggregate([{"$unwind":"$hotspot"},
                                {"$sort":{"hotspot.surface_temperature_celcius":-1}},
                                {"$project":{"_id":0}},
                                {"$limit":10}
                               ])
for rec in data:
    pprint(rec)

## f) Finding the number of fires in each day

In [ ]:
data = clim_hotspot.aggregate([{"$unwind":"$hotspot"},
                                {"$group":{"_id":"$date","count":{"$sum":1}}}                            
                               ])
for rec in data: 
    pprint(rec)

## g) Finding the average surface temp for each day

In [ ]:
data = clim_hotspot.aggregate([{"$unwind":"$hotspot"},
                                {"$group":{"_id":"$date","avg_surf_temp":{"$avg":"$hotspot.surface_temperature_celcius"}}}                            
                               ])
for rec in data:
    pprint(rec)